In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,294 kB]


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
#filter the dataset for total_votes being greater than or equal to 20
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
#vine_df.show()
vote_count = df.filter("total_votes >= 20")
#vote_count.count()

In [6]:
#create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
new_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
new_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [7]:
#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
new_table.filter(new_table["vine"] == "Y").count()

94

In [8]:
#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
new_table.filter(new_table["vine"] == "N").count()

40471

In [9]:
# number of five star reviews
five_star = new_table.filter(new_table["star_rating"]== 5)
#five_star.cache()

In [10]:
five_star.count()

15711

In [11]:
#total reviews
new_table.count()

40565

In [12]:
#percentage of five star reviews
five_star.count() / new_table.count()

0.3873043263897449

In [13]:
#percentage of paid reviews
five_star.filter(five_star["verified_purchase"] == "Y").count()/new_table.filter(new_table["verified_purchase"]== "Y").count()

0.38214921828616666

In [14]:
#percentage of unpaid reviews
five_star.filter(five_star["verified_purchase"] == "N").count()/new_table.filter(new_table["verified_purchase"]== "N").count()

0.3890147411274172

In [15]:
five_star.filter(five_star["vine"]=="Y").count()

48

In [16]:
five_star.filter(five_star["vine"]=="N").count()

15663